<a href="https://colab.research.google.com/github/flpsantos3/covid_pt/blob/main/casos_covid_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import datetime

**Sources:**

* vax - https://www.kaggle.com/komalkhetlani/coronavirus-covid19-vaccinations
* vax2 - https://www.ecdc.europa.eu/en/publications-data/data-covid-19-vaccination-eu-eea
* cases - https://www.kaggle.com/hussainaliarif/largest-covid19-world-dataset
* cases_2 - https://www.ecdc.europa.eu/en/publications-data/data-daily-new-cases-covid-19-eueea-country
* hosp - https://www.ecdc.europa.eu/en/publications-data/download-data-hospital-and-icu-admission-rates-and-current-occupancy-covid-19
* vars - https://www.ecdc.europa.eu/en/publications-data/data-virus-variants-covid-19-eueea

In [ ]:
vax=pd.read_csv('/content/drive/MyDrive/Datasets/world_vax.csv')
vax2=pd.read_csv('https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv')
cases=pd.read_csv('/content/drive/MyDrive/Datasets/Covid-19-World-Dataset.csv')
vars=pd.read_csv('https://opendata.ecdc.europa.eu/covid19/virusvariant/csv/data.csv')
hosp=pd.read_csv('https://opendata.ecdc.europa.eu/covid19/hospitalicuadmissionrates/csv/data.csv')
cases_2=pd.read_csv('https://opendata.ecdc.europa.eu/covid19/nationalcasedeath_eueea_daily_ei/csv/data.csv')

# Vaccines

In [ ]:
vax=vax.drop([0]).reset_index(drop=True)
vax_pt=vax.loc[(vax.location=='Portugal')].sort_values(by='date').reset_index(drop=True)
vax_pt.tail()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(vax_pt.date, vax_pt.people_vaccinated, label='First Dose & Full')
plt.plot(vax_pt.date, vax_pt.people_fully_vaccinated, label='Fully Vaxed')
plt.legend(loc='upper left')
plt.grid()
plt.xticks(range(0, len(vax_pt.date), 15))
plt.tight_layout()
plt.title('Total Number of People Vaccinated in Portugal')
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(vax_pt.date, vax_pt.daily_vaccinations, label='Daily Doses')
plt.legend(loc='upper left')
plt.grid()
plt.xticks(range(0, len(vax_pt.date), 15))
plt.tight_layout()
plt.title('Daily Vaccine Doses Administred in Portugal')
plt.show()

In [ ]:
vax_pt.daily_vaccinations=vax_pt.daily_vaccinations.astype(float) # daily_vax is object
vax_pt[['date','daily_vaccinations']].loc[vax_pt.daily_vaccinations==vax_pt.daily_vaccinations.max()]

# Cases & Deaths

In [ ]:
cases_pt=cases[cases.location=='Portugal'].sort_values(by='date').reset_index(drop=True)
cases_pt.head(3)

In [ ]:
# convert datetime to YYYY-MM-DD format
new_dates=[]
dates=list(cases_pt.date)
for val in dates:
  d=datetime.datetime.strptime(val, '%m/%d/%Y')
  new_dates.append(d.strftime('%Y-%m-%d'))

cases_pt['date']=new_dates
cases_pt.sort_values(by='date', inplace=True)
cases_pt.tail()

In [ ]:
cases_pt2=cases_2[cases_2.countriesAndTerritories=='Portugal'].reset_index(drop=True)

dates=list(cases_pt2.dateRep)
new_dates=[]
for val in dates:
  d=datetime.datetime.strptime(val, '%d/%m/%Y')
  new_dates.append(d.strftime('%Y-%m-%d'))

cases_pt2['dateRep']=new_dates
cases_pt2=cases_pt2.sort_values(by='dateRep').reset_index(drop=True)
cases_pt2=cases_pt2.rename(columns={'dateRep':'date',
                            'cases':'new_cases',
                            'deaths':'new_deaths'})
cases_pt2.tail(3)

In [ ]:
cases1=cases_pt[['date', 'new_cases','new_deaths']]
cases2=cases_pt2[['date', 'new_cases','new_deaths']]
cases2=cases2[cases2.date>'2021-06-29']

In [ ]:
new_cases=pd.concat([cases1, cases2], ignore_index=True)
new_cases.tail(3)

In [ ]:
plt.figure(figsize=(24,8))
plt.plot(new_cases.date, new_cases.new_cases, label='New Cases')
plt.legend(loc='upper left')
plt.grid()
plt.xticks(range(0, len(new_cases.date), 30))
plt.tight_layout()
plt.title('Evolution of Daily Covid Cases in Portugal')
plt.show()

In [ ]:
new_cases[['date','new_cases']].loc[new_cases.new_cases==new_cases.new_cases.max()]

In [ ]:
plt.figure(figsize=(24,8))
plt.plot(new_cases.date, new_cases.new_deaths, label='Deaths')
plt.legend(loc='upper left')
plt.grid()
plt.xticks(range(0, len(new_cases.date), 35))
plt.tight_layout()
plt.title('Evolution of Daily Deaths in Portugal')
plt.show()

In [ ]:
new_cases[['date','new_deaths']].loc[new_cases.new_deaths==new_cases.new_deaths.max()]

# Hospitalizations

In [ ]:
hosp_pt=hosp.loc[
                 (hosp.country=='Portugal') & 
                 ((hosp.indicator=='Daily hospital occupancy'))
                 ].sort_values(by='date').reset_index(drop=True)
hosp_pt.head(3)

In [ ]:
icu_pt=hosp.loc[
                 (hosp.country=='Portugal') & 
                 (hosp.indicator=='Daily ICU occupancy')
                 ].sort_values(by='date').reset_index(drop=True)
icu_pt.head(3)

In [ ]:
hosp_pt['icu']=icu_pt.value
hosp_pt.head(3)

In [ ]:
plt.figure(figsize=(24,8))
plt.plot(hosp_pt.date, hosp_pt.value, label='Hospitalized')
plt.plot(hosp_pt.date, hosp_pt.icu, label='ICU')
plt.legend(loc='upper left')
plt.grid()
plt.xticks(range(0, len(hosp_pt.date), 35))
plt.tight_layout()
plt.title('Number of Hospitalized Patients in Portugal')
plt.show()

In [ ]:
hosp_pt[['date','value']].loc[hosp_pt.value==hosp_pt.value.max()]

In [ ]:
hosp_pt[['date','icu']].loc[hosp_pt.icu==hosp_pt.icu.max()]